# Day 09. Exercise 02
# Metrics

### Запуск контейнера с нужными версиями

docker run -d \
  --platform linux/amd64 \
  -p 8888:8888 \
  -v $(pwd):/home/jovyan/work \
  --name sklearn \
  jupyter/scipy-notebook:python-3.8 \
  bash -c "pip install scikit-learn==0.23.1 && start-notebook.sh --NotebookApp.token=''"

#### и выбираем правильный kernel в vscode на localhost (который отдает докер)

In [1]:
import sys
print("Python версия:", sys.version)

import sklearn
print("scikit-learn версия:", sklearn.__version__)

import pandas as pd
print("pandas версия:", pd.__version__)

import numpy as np
print("numpy версия:", np.__version__)

Python версия: 3.8.13 | packaged by conda-forge | (default, Mar 25 2022, 06:04:10) 
[GCC 10.3.0]
scikit-learn версия: 0.23.1
pandas версия: 1.5.0
numpy версия: 1.23.3


## 0. Imports

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
import joblib
import json

## 1. Preprocessing

1. Create the same dataframe as in the previous exercise.
2. Using `train_test_split` with parameters `test_size=0.2`, `random_state=21` get `X_train`, `y_train`, `X_test`, `y_test`. Use the additional parameter `stratify`.

In [3]:
df = pd.read_csv('work/src/data/day-of-week-not-scaled.csv')
print(f"Размерность данных: {df.shape}")
print(f"Колонки в файле: {df.columns.tolist()}")

Размерность данных: (1686, 43)
Колонки в файле: ['numTrials', 'hour', 'uid_user_0', 'uid_user_1', 'uid_user_10', 'uid_user_11', 'uid_user_12', 'uid_user_13', 'uid_user_14', 'uid_user_15', 'uid_user_16', 'uid_user_17', 'uid_user_18', 'uid_user_19', 'uid_user_2', 'uid_user_20', 'uid_user_21', 'uid_user_22', 'uid_user_23', 'uid_user_24', 'uid_user_25', 'uid_user_26', 'uid_user_27', 'uid_user_28', 'uid_user_29', 'uid_user_3', 'uid_user_30', 'uid_user_31', 'uid_user_4', 'uid_user_6', 'uid_user_7', 'uid_user_8', 'labname_code_rvw', 'labname_lab02', 'labname_lab03', 'labname_lab03s', 'labname_lab05s', 'labname_laba04', 'labname_laba04s', 'labname_laba05', 'labname_laba06', 'labname_laba06s', 'labname_project1']


In [4]:
df_target = pd.read_csv('work/src/data/dayofweek.csv')

df['dayofweek'] = df_target['dayofweek']

print(f"Новая размерность: {df.shape}")
print("Финальные колонки:", df.columns.tolist())

df.head()

Новая размерность: (1686, 44)
Финальные колонки: ['numTrials', 'hour', 'uid_user_0', 'uid_user_1', 'uid_user_10', 'uid_user_11', 'uid_user_12', 'uid_user_13', 'uid_user_14', 'uid_user_15', 'uid_user_16', 'uid_user_17', 'uid_user_18', 'uid_user_19', 'uid_user_2', 'uid_user_20', 'uid_user_21', 'uid_user_22', 'uid_user_23', 'uid_user_24', 'uid_user_25', 'uid_user_26', 'uid_user_27', 'uid_user_28', 'uid_user_29', 'uid_user_3', 'uid_user_30', 'uid_user_31', 'uid_user_4', 'uid_user_6', 'uid_user_7', 'uid_user_8', 'labname_code_rvw', 'labname_lab02', 'labname_lab03', 'labname_lab03s', 'labname_lab05s', 'labname_laba04', 'labname_laba04s', 'labname_laba05', 'labname_laba06', 'labname_laba06s', 'labname_project1', 'dayofweek']


,numTrials,hour,uid_user_0,uid_user_1,uid_user_10,uid_user_11,uid_user_12,uid_user_13,uid_user_14,uid_user_15,...,labname_lab03,labname_lab03s,labname_lab05s,labname_laba04,labname_laba04s,labname_laba05,labname_laba06,labname_laba06s,labname_project1,dayofweek
0,1,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
1,2,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
2,3,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
3,4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
4,5,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4


In [5]:
print("Распределение целевой переменной:")
df['dayofweek'].value_counts().sort_index()

Распределение целевой переменной:


0    136
1    274
2    149
3    396
4    104
5    271
6    356
Name: dayofweek, dtype: int64

In [6]:
print(f"Общая информация о данных:")
print(f"- Количество образцов: {len(df)}")
print(f"- Количество признаков: {len(df.columns) - 1}")
print(f"- Количество классов: {df['dayofweek'].nunique()}")
print(f"- Классы: {sorted(df['dayofweek'].unique())}")

Общая информация о данных:
- Количество образцов: 1686
- Количество признаков: 43
- Количество классов: 7
- Классы: [0, 1, 2, 3, 4, 5, 6]


In [7]:
X = df.drop('dayofweek', axis=1)
y = df['dayofweek']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=21, 
    stratify=y
)

print("Размерность тренировочного набора:", X_train.shape)
print("Размерность тестового набора:", X_test.shape)
print(f"Размерность целевой переменной: {y.shape}")
print(f"Классы: {sorted(y.unique())}")

Размерность тренировочного набора: (1348, 43)
Размерность тестового набора: (338, 43)
Размерность целевой переменной: (1686,)
Классы: [0, 1, 2, 3, 4, 5, 6]


## 2. SVM

1. Use the best parameters from the previous exercise and train the model of SVM.
2. You need to calculate `accuracy`, `precision`, `recall`, `ROC AUC`.

 - `precision` and `recall` should be calculated for each class (use `average='weighted'`)
 - `ROC AUC` should be calculated for each class against any other class (all possible pairwise combinations) and then weighted average should be applied for the final metric
 - the code in the cell should display the result as below:

```
accuracy is 0.88757
precision is 0.89267
recall is 0.88757
roc_auc is 0.97878
```

### predict_proba() - возвращает вероятность для каждого класса

### Recall показывает: из всех реальных положительных случаев, сколько процентов мы нашли
```
Формула: Recall = TP / (TP + FN)

TP (True Positive) - правильно предсказали положительный класс
FN (False Negative) - пропустили положительный класс
```

#### Когда важен высокий recall?

- Медицинская диагностика (не пропустить болезнь)
- Детекция мошенничества
- Поиск террористов
- Лучше "ложная тревога", чем пропуск!

### ROC AUC
```
ROC - кривая, AUC - площадь под этой кривой.
ROC кривая строится так:

Берем разные пороги (0.1, 0.2, 0.3... 0.9)
Для каждого порога считаем:

True Positive Rate (Recall)
False Positive Rate (сколько неправильно классифицировали как положительные)


Рисуем график: FPR по X, TPR по Y

AUC (Area Under Curve) - площадь под ROC кривой:

AUC = 1.0 - идеальная модель
AUC = 0.5 - случайное угадывание
AUC > 0.8 - хорошая модель
```

#### Зачем ROC AUC?

- Показывает качество модели независимо от порога
- Можно сравнивать модели между собой
- Учитывает predict_proba, а не только финальные предсказания

- ovo = one-vs-one = все возможные парные комбинации классов
- ovr = one-vs-rest = каждый класс против всех остальных

In [8]:
svm_best_params = {'probability': True, 'random_state': 21, 'C': 10.0, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf'}

svm_model = SVC(**svm_best_params)
svm_model.fit(X_train, y_train)

y_pred_svm = svm_model.predict(X_test)
y_pred_proba_svm = svm_model.predict_proba(X_test)

svm_accuracy = accuracy_score(y_test, y_pred_svm)
svm_precision = precision_score(y_test, y_pred_svm, average='weighted')
svm_recall = recall_score(y_test, y_pred_svm, average='weighted')
svm_roc_auc = roc_auc_score(y_test, y_pred_proba_svm, multi_class='ovo', average='weighted')

print(f"accuracy is {svm_accuracy:.5f}")
print(f"precision is {svm_precision:.5f}")
print(f"recall is {svm_recall:.5f}")
print(f"roc_auc is {svm_roc_auc:.5f}")

accuracy is 0.88757
precision is 0.89267
recall is 0.88757
roc_auc is 0.97878


## 3. Decision tree

1. The same task for decision tree

In [9]:
tree_best_params = {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 21, 'random_state': 21}

tree_model = DecisionTreeClassifier(**tree_best_params)
tree_model.fit(X_train, y_train)

y_pred_tree = tree_model.predict(X_test)
y_pred_proba_tree = tree_model.predict_proba(X_test)

tree_accuracy = accuracy_score(y_test, y_pred_tree)
tree_precision = precision_score(y_test, y_pred_tree, average='weighted')
tree_recall = recall_score(y_test, y_pred_tree, average='weighted')
tree_roc_auc = roc_auc_score(y_test, y_pred_proba_tree, multi_class='ovo', average='weighted')

print(f"accuracy is {tree_accuracy:.5f}")
print(f"precision is {tree_precision:.5f}")
print(f"recall is {tree_recall:.5f}")
print(f"roc_auc is {tree_roc_auc:.5f}")

accuracy is 0.88462
precision is 0.88765
recall is 0.88462
roc_auc is 0.93528


## 4. Random forest

1. The same task for random forest.

In [10]:
rf_best_params = {'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 24, 'n_estimators': 100, 'random_state': 21}

rf_model = RandomForestClassifier(**rf_best_params)
rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)
y_pred_proba_rf = rf_model.predict_proba(X_test)

rf_accuracy = accuracy_score(y_test, y_pred_rf)
rf_precision = precision_score(y_test, y_pred_rf, average='weighted')
rf_recall = recall_score(y_test, y_pred_rf, average='weighted')
rf_roc_auc = roc_auc_score(y_test, y_pred_proba_rf, multi_class='ovo', average='weighted')

print(f"accuracy is {rf_accuracy:.5f}")
print(f"precision is {rf_precision:.5f}")
print(f"recall is {rf_recall:.5f}")
print(f"roc_auc is {rf_roc_auc:.5f}")

accuracy is 0.92604
precision is 0.92754
recall is 0.92604
roc_auc is 0.98939


## 5. Predictions

1. Choose the best model.
2. Analyze: for which `weekday` your model makes the most errors (in % of the total number of samples of that class in your full dataset), for which `labname` and for which `users`.
3. Save the model.

In [11]:
print("=== СРАВНЕНИЕ МОДЕЛЕЙ ===")

model_results = {
    'SVM': {
        'accuracy': svm_accuracy,
        'precision': svm_precision,
        'recall': svm_recall,
        'roc_auc': svm_roc_auc,
        'model': svm_model,
        'predictions': y_pred_svm
    },
    'Decision Tree': {
        'accuracy': tree_accuracy,
        'precision': tree_precision,
        'recall': tree_recall,
        'roc_auc': tree_roc_auc,
        'model': tree_model,
        'predictions': y_pred_tree
    },
    'Random Forest': {
        'accuracy': rf_accuracy,
        'precision': rf_precision,
        'recall': rf_recall,
        'roc_auc': rf_roc_auc,
        'model': rf_model,
        'predictions': y_pred_rf
    }
}

for model_name, results in model_results.items():
    print(f"{model_name}:")
    print(f"  accuracy: {results['accuracy']:.5f}")
    print(f"  precision: {results['precision']:.5f}")
    print(f"  recall: {results['recall']:.5f}")
    print(f"  roc_auc: {results['roc_auc']:.5f}")
    print()

best_model_name = max(model_results.keys(), key=lambda x: model_results[x]['roc_auc'])
best_model = model_results[best_model_name]['model']
best_predictions = model_results[best_model_name]['predictions']
best_roc_auc = model_results[best_model_name]['roc_auc']

print(f"Лучшая модель: {best_model_name} с ROC AUC: {best_roc_auc:.5f}")

=== СРАВНЕНИЕ МОДЕЛЕЙ ===
SVM:
  accuracy: 0.88757
  precision: 0.89267
  recall: 0.88757
  roc_auc: 0.97878

Decision Tree:
  accuracy: 0.88462
  precision: 0.88765
  recall: 0.88462
  roc_auc: 0.93528

Random Forest:
  accuracy: 0.92604
  precision: 0.92754
  recall: 0.92604
  roc_auc: 0.98939

Лучшая модель: Random Forest с ROC AUC: 0.98939


In [12]:
print("=== АНАЛИЗ ОШИБОК ПО ДНЯМ НЕДЕЛИ ===")

weekday_names = ['Понедельник', 'Вторник', 'Среда', 'Четверг', 'Пятница', 'Суббота', 'Воскресенье']

weekday_errors = {}
for weekday in sorted(y.unique()):
    mask_true = y_test == weekday
    mask_pred_wrong = (y_test == weekday) & (best_predictions != weekday)
    
    total_in_test = mask_true.sum()
    errors_in_test = mask_pred_wrong.sum()
    
    if total_in_test > 0:
        error_rate = errors_in_test / total_in_test * 100
        weekday_errors[weekday] = error_rate
        
        day_name = weekday_names[weekday] if weekday < len(weekday_names) else f'День {weekday}'
        print(f"{day_name} (класс {weekday}): {error_rate:.2f}% ошибок ({errors_in_test}/{total_in_test} ошибок)")

worst_weekday = max(weekday_errors, key=weekday_errors.get)
worst_weekday_error_rate = weekday_errors[worst_weekday]
worst_weekday_name = weekday_names[worst_weekday] if worst_weekday < len(weekday_names) else f'День {worst_weekday}'

print(f"\nДень недели с наибольшими ошибками: {worst_weekday_name} ({worst_weekday_error_rate:.2f}% ошибок)")

=== АНАЛИЗ ОШИБОК ПО ДНЯМ НЕДЕЛИ ===
Понедельник (класс 0): 22.22% ошибок (6/27 ошибок)
Вторник (класс 1): 7.27% ошибок (4/55 ошибок)
Среда (класс 2): 6.67% ошибок (2/30 ошибок)
Четверг (класс 3): 3.75% ошибок (3/80 ошибок)
Пятница (класс 4): 14.29% ошибок (3/21 ошибок)
Суббота (класс 5): 9.26% ошибок (5/54 ошибок)
Воскресенье (класс 6): 2.82% ошибок (2/71 ошибок)

День недели с наибольшими ошибками: Понедельник (22.22% ошибок)


In [13]:
print("=== АНАЛИЗ ОШИБОК ПО LABNAME ===")

test_indices = X_test.index
df_test_analysis = df.loc[test_indices].copy()
df_test_analysis['predictions'] = best_predictions

labname_columns = [col for col in df.columns if col.startswith('labname_')]
print(f"Найдено {len(labname_columns)} labname колонок: {labname_columns}")

def get_original_labname(row):
    for col in labname_columns:
        if row[col] == 1:
            return col.replace('labname_', '')
    return 'unknown'

df_test_analysis['original_labname'] = df_test_analysis.apply(get_original_labname, axis=1)

labname_errors = {}
print(f"\n{'Labname':<15} {'Ошибки':<8} {'Всего':<8} {'Процент ошибок':<15}")
print(f"{'-'*15} {'-'*8} {'-'*8} {'-'*15}")

for labname in sorted(df_test_analysis['original_labname'].unique()):
    mask_lab = df_test_analysis['original_labname'] == labname
    if mask_lab.sum() > 0:
        errors = (df_test_analysis.loc[mask_lab, 'dayofweek'] != df_test_analysis.loc[mask_lab, 'predictions']).sum()
        total = mask_lab.sum()
        error_rate = errors / total * 100
        labname_errors[labname] = error_rate
        print(f"{labname:<15} {errors:<8} {total:<8} {error_rate:<15.2f}%")

if labname_errors:
    worst_labname = max(labname_errors, key=labname_errors.get)
    worst_labname_error_rate = labname_errors[worst_labname]
    
    best_labname = min(labname_errors, key=labname_errors.get)
    best_labname_error_rate = labname_errors[best_labname]
    
    print(f"\n❌ Labname с наибольшими ошибками: {worst_labname} ({worst_labname_error_rate:.2f}% ошибок)")
    print(f"✅ Labname с наименьшими ошибками: {best_labname} ({best_labname_error_rate:.2f}% ошибок)")

=== АНАЛИЗ ОШИБОК ПО LABNAME ===
Найдено 11 labname колонок: ['labname_code_rvw', 'labname_lab02', 'labname_lab03', 'labname_lab03s', 'labname_lab05s', 'labname_laba04', 'labname_laba04s', 'labname_laba05', 'labname_laba06', 'labname_laba06s', 'labname_project1']

Labname         Ошибки   Всего    Процент ошибок 
--------------- -------- -------- ---------------
code_rvw        1        13       7.69           %
lab03           1        1        100.00         %
lab03s          0        1        0.00           %
lab05s          1        6        16.67          %
laba04          6        35       17.14          %
laba04s         2        25       8.00           %
laba05          1        47       2.13           %
laba06          1        9        11.11          %
laba06s         2        15       13.33          %
project1        10       186      5.38           %

❌ Labname с наибольшими ошибками: lab03 (100.00% ошибок)
✅ Labname с наименьшими ошибками: lab03s (0.00% ошибок)


In [14]:
print("=== АНАЛИЗ ОШИБОК ПО ПОЛЬЗОВАТЕЛЯМ (ТОП-10) ===")

uid_columns = [col for col in df.columns if col.startswith('uid_user_')]
print(f"Найдено {len(uid_columns)} uid колонок")

def get_original_uid(row):
    for col in uid_columns:
        if row[col] == 1:
            return col.replace('uid_user_', '')
    return 'unknown'

df_test_analysis['original_uid'] = df_test_analysis.apply(get_original_uid, axis=1)

user_errors = []
for uid in sorted(df_test_analysis['original_uid'].unique()):
    mask_user = df_test_analysis['original_uid'] == uid
    if mask_user.sum() > 0:
        errors = (df_test_analysis.loc[mask_user, 'dayofweek'] != df_test_analysis.loc[mask_user, 'predictions']).sum()
        total = mask_user.sum()
        error_rate = errors / total * 100 if total > 0 else 0
        user_errors.append((uid, error_rate, errors, total))

user_errors.sort(key=lambda x: x[1], reverse=True)

print("\nТоп-10 пользователей с наибольшим процентом ошибок:")
print(f"{'Ранг':<5} {'User ID':<8} {'Ошибки':<8} {'Всего':<8} {'Процент ошибок':<15}")
print(f"{'-'*5} {'-'*8} {'-'*8} {'-'*8} {'-'*15}")

for i, (uid, error_rate, errors, total) in enumerate(user_errors[:10]):
    print(f"{i+1:<5} {uid:<8} {errors:<8} {total:<8} {error_rate:<15.2f}%")

if user_errors:
    worst_user = user_errors[0]
    print(f"\n❌ Пользователь с наибольшими ошибками: User {worst_user[0]} ({worst_user[1]:.2f}% ошибок)")
    
    user_errors_sorted_asc = sorted(user_errors, key=lambda x: x[1])
    best_user = user_errors_sorted_asc[0]
    if best_user[1] < worst_user[1]:
        print(f"✅ Пользователь с наименьшими ошибками: User {best_user[0]} ({best_user[1]:.2f}% ошибок)")

=== АНАЛИЗ ОШИБОК ПО ПОЛЬЗОВАТЕЛЯМ (ТОП-10) ===
Найдено 30 uid колонок

Топ-10 пользователей с наибольшим процентом ошибок:
Ранг  User ID  Ошибки   Всего    Процент ошибок 
----- -------- -------- -------- ---------------
1     22       1        1        100.00         %
2     6        2        4        50.00          %
3     16       1        5        20.00          %
4     18       1        6        16.67          %
5     27       1        6        16.67          %
6     3        2        14       14.29          %
7     30       1        8        12.50          %
8     31       2        18       11.11          %
9     2        3        28       10.71          %
10    19       2        19       10.53          %

❌ Пользователь с наибольшими ошибками: User 22 (100.00% ошибок)
✅ Пользователь с наименьшими ошибками: User 1 (0.00% ошибок)


In [15]:
model_filename = f'work/src/ex02/model/best_model_metrics.joblib'
joblib.dump(best_model, model_filename)
print(f"Лучшая модель ({best_model_name}) сохранена как: {model_filename}")

model_info = {
    'model_type': best_model_name,
    'best_roc_auc': best_roc_auc,
    'best_accuracy': model_results[best_model_name]['accuracy'],
    'best_precision': model_results[best_model_name]['precision'],
    'best_recall': model_results[best_model_name]['recall'],
    'worst_weekday': worst_weekday_name,
    'worst_weekday_error_rate': worst_weekday_error_rate,
    'worst_labname': worst_labname,
    'worst_labname_error_rate': worst_labname_error_rate,
    'parameters': str(best_model.get_params())
}

with open('work/src/ex02/model/model_info_metrics.json', 'w') as f:
    json.dump(model_info, f, indent=2)

Лучшая модель (Random Forest) сохранена как: work/src/ex02/model/best_model_metrics.joblib


## 6. Function

1. Write a function that takes a list of different models and a corresponding list of parameters (dicts) and returns a dict that contains all the 4 metrics for each model.

In [19]:
def evaluate_models_metrics(models_list, params_list, X_train, y_train, X_test, y_test):
    """
    Оценка списка моделей с соответствующими параметрами
    
    Args:
        models_list: список классов моделей (например, [SVC, DecisionTreeClassifier, RandomForestClassifier])
        params_list: список словарей параметров для каждой модели
        X_train, y_train: тренировочные данные
        X_test, y_test: тестовые данные
    
    Returns:
        dict: словарь с названиями моделей как ключами и метриками как значениями
    """
    results = {}
    
    for i, (model_class, params) in enumerate(zip(models_list, params_list)):
        model_name = model_class.__name__
        print(f"Оцениваем {model_name}...")
        
        try:
            if model_class == SVC:
                model_params = params.copy()
                model_params['probability'] = True
                if 'random_state' not in model_params:
                    model_params['random_state'] = 21
                model = model_class(**model_params)
            else:
                model_params = params.copy()
                if 'random_state' not in model_params:
                    model_params['random_state'] = 21
                model = model_class(**model_params)
            
            model.fit(X_train, y_train)
            
            y_pred = model.predict(X_test)
            y_pred_proba = model.predict_proba(X_test)
            
            accuracy = accuracy_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred, average='weighted')
            recall = recall_score(y_test, y_pred, average='weighted')
            roc_auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovo', average='weighted')
            
            results[model_name] = {
                'accuracy': accuracy,
                'precision': precision,
                'recall': recall,
                'roc_auc': roc_auc
            }
            
            print(f"  accuracy: {accuracy:.5f}")
            print(f"  precision: {precision:.5f}")
            print(f"  recall: {recall:.5f}")
            print(f"  roc_auc: {roc_auc:.5f}")
            print()
            
        except Exception as e:
            print(f"Ошибка при оценке {model_name}: {str(e)}")
            results[model_name] = {
                'accuracy': 0.0,
                'precision': 0.0,
                'recall': 0.0,
                'roc_auc': 0.0
            }
    
    return results

In [20]:
print("=== ТЕСТИРОВАНИЕ ФУНКЦИИ С ЛУЧШИМИ ПАРАМЕТРАМИ ===")

test_models = [SVC, DecisionTreeClassifier, RandomForestClassifier]
test_params = [
    svm_best_params,
    tree_best_params,
    rf_best_params
]

function_results = evaluate_models_metrics(test_models, test_params, X_train, y_train, X_test, y_test)

=== ТЕСТИРОВАНИЕ ФУНКЦИИ С ЛУЧШИМИ ПАРАМЕТРАМИ ===
Оцениваем SVC...
  accuracy: 0.88757
  precision: 0.89267
  recall: 0.88757
  roc_auc: 0.97878

Оцениваем DecisionTreeClassifier...
  accuracy: 0.88462
  precision: 0.88765
  recall: 0.88462
  roc_auc: 0.93528

Оцениваем RandomForestClassifier...
  accuracy: 0.92604
  precision: 0.92754
  recall: 0.92604
  roc_auc: 0.98939



In [22]:
print("=== РЕЗУЛЬТАТЫ ФУНКЦИИ ===")
for model_name, metrics in function_results.items():
    print(f"{model_name}:")
    for metric_name, value in metrics.items():
        print(f"  {metric_name}: {value:.5f}")
    print()

=== РЕЗУЛЬТАТЫ ФУНКЦИИ ===
SVC:
  accuracy: 0.88757
  precision: 0.89267
  recall: 0.88757
  roc_auc: 0.97878

DecisionTreeClassifier:
  accuracy: 0.88462
  precision: 0.88765
  recall: 0.88462
  roc_auc: 0.93528

RandomForestClassifier:
  accuracy: 0.92604
  precision: 0.92754
  recall: 0.92604
  roc_auc: 0.98939



In [23]:
print("=== СРАВНЕНИЕ РЕЗУЛЬТАТОВ ФУНКЦИИ И УПРАЖНЕНИЯ ===")

model_name_mapping = {
    'SVC': 'SVM',
    'DecisionTreeClassifier': 'Decision Tree', 
    'RandomForestClassifier': 'Random Forest'
}

print(f"{'Модель':<20} {'Метрика':<12} {'Упражнение':<12} {'Функция':<12} {'Разница':<12}")
print("-" * 70)

for func_model_name, func_metrics in function_results.items():
    orig_model_name = model_name_mapping[func_model_name]
    orig_metrics = model_results[orig_model_name]
    
    for metric in ['accuracy', 'precision', 'recall', 'roc_auc']:
        orig_value = orig_metrics[metric]
        func_value = func_metrics[metric]
        diff = abs(orig_value - func_value)
        
        print(f"{orig_model_name:<20} {metric:<12} {orig_value:<12.5f} {func_value:<12.5f} {diff:<12.6f}")
    print()

all_match = True
tolerance = 1e-5

for func_model_name, func_metrics in function_results.items():
    orig_model_name = model_name_mapping[func_model_name]
    orig_metrics = model_results[orig_model_name]
    
    for metric in ['accuracy', 'precision', 'recall', 'roc_auc']:
        if abs(orig_metrics[metric] - func_metrics[metric]) > tolerance:
            all_match = False
            break

if all_match:
    print("✅ РЕЗУЛЬТАТЫ СОВПАДАЮТ! Функция работает корректно.")
else:
    print("❌ Есть расхождения в результатах. Проверьте реализацию функции.")

=== СРАВНЕНИЕ РЕЗУЛЬТАТОВ ФУНКЦИИ И УПРАЖНЕНИЯ ===
Модель               Метрика      Упражнение   Функция      Разница     
----------------------------------------------------------------------
SVM                  accuracy     0.88757      0.88757      0.000000    
SVM                  precision    0.89267      0.89267      0.000000    
SVM                  recall       0.88757      0.88757      0.000000    
SVM                  roc_auc      0.97878      0.97878      0.000000    

Decision Tree        accuracy     0.88462      0.88462      0.000000    
Decision Tree        precision    0.88765      0.88765      0.000000    
Decision Tree        recall       0.88462      0.88462      0.000000    
Decision Tree        roc_auc      0.93528      0.93528      0.000000    

Random Forest        accuracy     0.92604      0.92604      0.000000    
Random Forest        precision    0.92754      0.92754      0.000000    
Random Forest        recall       0.92604      0.92604      0.000000    
